In [ ]:
import os 
import cv2
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader 
import pathlib
import glob

### mounting google drive on colab

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### extracting zip file

In [ ]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/nature_12K.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/nature') #Extracts the files into the /nature folder
zip_ref.close()

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:


from torch.utils.data.sampler import SubsetRandomSampler

# how many samples per batch to load
batch_size = 64
# percentage of training set to use as validation
valid_size = 0.2

#To unzip dataset in colab
data_dir='/nature/inaturalist_12K'

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(200),
                                       transforms.RandomHorizontalFlip(),

                                       transforms.ToTensor(),
                                       
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(200),
                                     
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = torchvision.datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = torchvision.datasets.ImageFolder(data_dir + '/val', transform=test_transforms)
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
print(len(valid_loader))


In [ ]:
train_path='/nature/inaturalist_12K/train'
test_path='/nature/inaturalist_12K/val'
print(len(train_loader))


In [ ]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])
classes=(classes[1:])
print(classes)

In [ ]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
train_count=int(train_count-train_count*0.2)             
test_count=len(glob.glob(test_path+'/**/*.jpg'))
val_count=int(train_count*0.2)

In [ ]:
# K=32
# S=3
# batch_norm=True
# activation='Relu'
# nodes=512
class ConvNet(nn.Module):
  def __init__(self,epoch,K,S,factor,activation,batch_norm,drop_out,nodes,num_class=10):
    
    super(ConvNet,self).__init__()
    
    self.batch_norm=batch_norm
    
    self.conv1=nn.Conv2d(in_channels=3,out_channels=K,kernel_size=S,stride=1,padding=(S-1)//2) #its output size will be=(batch=256,K,200,200)
    
    if self.batch_norm=='Yes':
      self.bn1=nn.BatchNorm2d(K)
      
    if activation=='Relu':
      
      self.activ1=nn.ReLU()
    elif activation=='Gelu':
    
      self.activ1=nn.GELU()
    elif activation=='Silu':
      
      self.activ1=nn.SiLU()
    elif activation=='Mish':
      
      self.activ1=nn.Mish()
      
    self.maxpool1=nn.MaxPool2d(kernel_size=2)## shape=(256,K,100,100)
    
    K_prev=K
    K=int(K*factor)
    #layer 2
    self.conv2=nn.Conv2d(in_channels=K_prev,out_channels=K,kernel_size=S,stride=1,padding=(S-1)//2) #its output size will be=(batch=256,K,100,100)
    if self.batch_norm=='Yes':
      self.bn2=nn.BatchNorm2d(K)
    if activation=='Relu':
      self.activ2=nn.ReLU()
    elif activation=='Gelu':
      self.activ2=nn.GELU()
    elif activation=='Silu':
      self.activ2=nn.SiLU()
    elif activation=='Mish':
      self.activ2=nn.Mish()
    self.maxpool2=nn.MaxPool2d(kernel_size=2)
    ##shape=(256,K,50,50)
    K_prev=K
    K=int(K*factor)
    # layer 3
    self.conv3=nn.Conv2d(in_channels=K_prev,out_channels=K,kernel_size=S,stride=1,padding=(S-1)//2) #its output size will be=(batch=256,K,50,50)
    if self.batch_norm=='Yes':
      self.bn3=nn.BatchNorm2d(K)
    if activation=='Relu':
      self.activ3=nn.ReLU()
    elif activation=='Gelu':
      self.activ3=nn.GELU()
    elif activation=='Silu':
      self.activ3=nn.SiLU()
    elif activation=='Mish':
      self.activ3=nn.Mish()
    self.maxpool3=nn.MaxPool2d(kernel_size=2)
    ##shape=(256,K,25,25)
    K_prev=K
    K=int(K*factor)
    #layer 4
    self.conv4=nn.Conv2d(in_channels=K_prev,out_channels=K,kernel_size=S,stride=1,padding=(S-1)//2) #its output size will be=(batch=256,K,25,25)
    if self.batch_norm=='Yes':
      self.bn4=nn.BatchNorm2d(K)
    if activation=='Relu':
      self.activ4=nn.ReLU()
    elif activation=='Gelu':
      self.activ4=nn.GELU()
    elif activation=='Silu':
      self.activ4=nn.SiLU()
    elif activation=='Mish':
      self.activ4=nn.Mish()
    self.maxpool4=nn.MaxPool2d(kernel_size=2)
    ## shape=(256,K,12,12)
    K_prev=K
    K=int(K*factor)
    #layer 5
    self.conv5=nn.Conv2d(in_channels=K_prev,out_channels=K,kernel_size=S,stride=1,padding=(S-1)//2) #its output size will be=(batch=256,K,12,12)
    if self.batch_norm=='Yes':
      self.bn5=nn.BatchNorm2d(K)
    if activation=='Relu':
      self.activ5=nn.ReLU()
    elif activation=='Gelu':
      self.activ5=nn.GELU()
    elif activation=='Silu':
      self.activ5=nn.SiLU()
    elif activation=='Mish':
      self.activ5=nn.Mish()
    self.maxpool5=nn.MaxPool2d(kernel_size=2)
    ## shape(256,K,6,6)
    #print(len(self.maxpool5))
     ## shape(256,K,)
    ## FC layer
    self.dropout1=nn.Dropout(drop_out)            ### ADD DROPOUT HERE
    #print("D:",self.dropout1.shape)
    self.fc1=nn.Linear(in_features=6*6*K,out_features=nodes)
    self.dropout2=nn.Dropout(drop_out)   ### ADD DROP OUT HERE
    self.fc2=nn.Linear(in_features=nodes,out_features=10)
    self.K=K
    ##feed forward

  def forward(self,x):
    
    out=self.conv1(x)
    if self.batch_norm=='Yes':
      out=self.bn1(out)
    out=self.activ1(out)
    out=self.maxpool1(out)

    out=self.conv2(out)
    if self.batch_norm=='Yes':
      out=self.bn2(out)
    out=self.activ2(out)
    out=self.maxpool2(out)

    out=self.conv3(out)
    if self.batch_norm=='Yes':
      out=self.bn3(out)
    out=self.activ3(out)
    out=self.maxpool3(out)

    out=self.conv4(out)
    if self.batch_norm=='Yes':
      out=self.bn4(out)
    out=self.activ4(out)
    out=self.maxpool4(out)
    
    out=self.conv5(out)
    if self.batch_norm=='Yes':
      out=self.bn5(out)
    out=self.activ5(out)
    out=self.maxpool5(out)
    
    out=self.dropout1(out)
    b,c,d,e=out.shape
    
    out=out.view(-1,d*e*self.K)  ###### there is some problem
    
    out=self.fc1(out)
    out=self.dropout2(out)
    out=self.fc2(out)
    out=F.softmax(out,dim=1)
    
    return out


epoch,K,S,factor,activation,batch_norm,drop_out,nodes=2,32,3,1/2,'Relu','Yes',0.2,512
net=ConvNet(epoch,K,S,factor,activation,batch_norm,drop_out,nodes,num_class=10)
print(net)

In [ ]:
from torch.autograd import Variable
##model training

best_accuracy=0

for epoch in range(10):

  model.train()
  train_accuracy=0.0
  train_loss=0.0
  for i,(images,labels) in enumerate(train_loader):
    
    if torch.cuda.is_available():
      images=Variable(images.cuda())
      labels=Variable(labels.cuda())
    
    optimizer.zero_grad()
    
    outputs=model(images)
    loss=loss_fun(outputs,labels)
    #print(loss)
    loss.backward()
    optimizer.step()
    #print(loss.cpu().data)
    train_loss+=loss.cpu().data*images.size(0)
    _,prediction=torch.max(outputs.data,1)

    train_accuracy+=int(torch.sum(prediction==labels.data))
  train_accuracy=train_accuracy/train_count
  train_loss=train_loss/train_count

  ##evalution on testing data
  model.eval()

  test_accuracy=0.0

  for i,(images,labels) in enumerate(test_loader):
    if torch.cuda.is_available():
      images=Variable(images.cuda())
      labels=Variable(labels.cuda())
      
    outputs=model(images)

    _,prediction=torch.max(outputs.data,1)
    test_accuracy+=int(torch.sum(prediction==labels.data))
  
  test_accuracy=test_accuracy/test_count


  print('Epoch:'+str(epoch)+'Train Loss:'+str(int(train_loss))+'Train accuracy'+str(train_accuracy)+'Test accuracy'+str(test_accuracy))
  
  if test_accuracy>best_accuracy:
    torch.save(model.state_dict(),'best_checkpoint.model')
    best_accuracy=test_accuracy
